# SUPERT 

Example code for using the supert metrics

- Paper: https://arxiv.org/abs/2005.03724
- Git-Project: https://github.com/danieldeutsch/SUPERT

In [1]:
import sys
sys.path.append("SUPERT")
import time
import tracemalloc 

In [2]:
# to get the reader working
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dwarakvittal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dwarakvittal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from ref_free_metrics.supert import Supert
from utils.data_reader import CorpusReader
from utils.evaluator import evaluate_summary_rouge, add_result

In [4]:
# read docs and summaries
reader = CorpusReader('SUPERT/data/topic_1')
source_docs = reader()
summaries = reader.readSummaries() 
refs = reader.readReferences()

# Evaluate Summaries

### BERT

In [9]:
tracemalloc.start()      

In [10]:
bert_start = time.perf_counter()
# compute the Supert scores
supert = Supert(docs=source_docs, sentence_transformer='bert-base-nli-stsb-mean-tokens')
bert_scores = supert(summaries)
bert_end = time.perf_counter() - bert_start

[{'idx': 0, 'name': '0', 'path': '0_BERT', 'type': 'sentence_transformers.models.BERT'}, {'idx': 1, 'name': '1', 'path': '1_Pooling', 'type': 'sentence_transformers.models.Pooling'}]


In [11]:
current, peak = tracemalloc.get_traced_memory()   
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")   
tracemalloc.stop()

Current memory usage is 19.098954MB; Peak was 21.542339MB


In [12]:
print(bert_scores)
print("BERT calc time:", bert_end, "sec")

[0.8140881996572803, 0.8140881996572803]
BERT calc time: 22.370334291999825 sec


### RoBERTa

In [5]:
tracemalloc.start()      

In [6]:
roberta_start = time.perf_counter()
roberta_supert = Supert(docs=source_docs, sentence_transformer='roberta-base-nli-stsb-mean-tokens')
roberta_scores = roberta_supert(summaries)
roberta_end = time.perf_counter() - roberta_start

[{'idx': 0, 'name': '0', 'path': '0_Transformer', 'type': 'sentence_transformers.models.RoBERTa'}, {'idx': 1, 'name': '1', 'path': '1_Pooling', 'type': 'sentence_transformers.models.Pooling'}]


In [7]:
current, peak = tracemalloc.get_traced_memory()   
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")   
tracemalloc.stop()

Current memory usage is 40.70627MB; Peak was 45.134747MB


In [8]:
print(roberta_scores)
print("RoBERTa calc time:", roberta_end, "sec")

[0.8320928335930917, 0.8320928335930917]
RoBERTa calc time: 23.749510917 sec


### ALBERT

In [ ]:
albert_supert = Supert(docs=source_docs, sentence_transformer='paraphrase-albert-base-v2')
scores = albert_supert(summaries)

In [ ]:
print(scores)

### DistilBERT

In [5]:
tracemalloc.start()

In [6]:
distilbert_start = time.perf_counter()
distilbert_supert = Supert(docs=source_docs, sentence_transformer='distilbert-base-nli-stsb-mean-tokens')
distilbert_scores = distilbert_supert(summaries)
distilbert_end = time.perf_counter() - distilbert_start

[{'idx': 0, 'name': '0', 'path': '0_Transformer', 'type': 'sentence_transformers.models.DistilBERT'}, {'idx': 1, 'name': '1', 'path': '1_Pooling', 'type': 'sentence_transformers.models.Pooling'}]


In [7]:
current, peak = tracemalloc.get_traced_memory()   
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")   
tracemalloc.stop()

Current memory usage is 18.905437MB; Peak was 21.34021MB


In [8]:
print(distilbert_scores)
print("DistilBERT calc time:", distilbert_end, "sec")

[0.832189884651843, 0.832189884651843]
DistilBERT calc time: 10.834976207999999 sec


# Create Summaries with RL

### RL Class from other py-script

In [9]:
from ref_free_metrics.supert import Supert
from summariser.ngram_vector.vector_generator import Vectoriser
from summariser.deep_td import DeepTDAgent as RLAgent
from utils.data_reader import CorpusReader
from utils.evaluator import evaluate_summary_rouge, add_result

class RLSummarizer():
    def __init__(self,reward_func, reward_strict=5.,rl_strict=5.,train_episode=5000, base_length=200, sample_summ_num=5000, gpu=False):
        self.reward_func = reward_func
        self.reward_strict = reward_strict
        self.rl_strict = rl_strict
        self.train_episode = train_episode
        self.base_length = base_length
        self.sample_summ_num = sample_summ_num
        self.gpu = gpu

    def get_sample_summaries(self, docs, summ_max_len=100):
        vec = Vectoriser(docs,summ_max_len)
        summary_list = vec.sample_random_summaries(self.sample_summ_num)
        rewards = self.reward_func(summary_list)
        assert len(summary_list) == len(rewards)
        return summary_list, rewards

    def summarize(self, docs, summ_max_len=100):
        # generate sample summaries for memory replay
        summaries, rewards = self.get_sample_summaries(docs, summ_max_len)
        vec = Vectoriser(docs,base=self.base_length)
        rl_agent = RLAgent(vec, summaries, strict_para=self.rl_strict, train_round=self.train_episode, gpu=self.gpu)
        summary = rl_agent(rewards)
        return summary

In [10]:
# generate summaries using reinforcement learning, with supert as reward function
rl_summarizer = RLSummarizer(reward_func = distilbert_supert, gpu = False)
summary = rl_summarizer.summarize(source_docs, summ_max_len=100)

# print out the generated summary
print(summary)

neural-rl training episodes: 100%|██████████| 5000/5000 [00:38<00:00, 130.23it/s]


Alvarez is likely to face at least 10 counts of murder, the chief said. The driver of the SUV, identified as Juan Manuel Alvarez, 25, of Compton, Calif., was taken into custody, and police said he would be charged with homicide. A counterterrorism command post was set up near the crash site, but authorities determined within an hour that the disaster was almost certainly the work of one troubled man. He said he got on train 100 at 5:19 a.m. in Simi Valley. ``This is a complete outrage.'' He was told she was taken to the hospital and released. ...


In [5]:
summary = "Alvarez is likely to face at least 10 counts of murder, the chief said. The driver of the SUV, identified as Juan Manuel Alvarez, 25, of Compton, Calif., was taken into custody, and police said he would be charged with homicide. A counterterrorism command post was set up near the crash site, but authorities determined within an hour that the disaster was almost certainly the work of one troubled man. He said he got on train 100 at 5:19 a.m. in Simi Valley. ``This is a complete outrage.'' He was told she was taken to the hospital and released."

In [6]:
rouge_scores = {}
for ref in refs:
    rs = evaluate_summary_rouge(summary, ref)
    add_result(rouge_scores, rs)
mm = 'ROUGE-1'
print('reference-based',np.mean(rouge_scores[mm]),'\n',rouge_scores)

execute_rouge command is perl /Users/dwarakvittal/Documents/Projekte/NLP_Metric/SUPERT/rouge/ROUGE-RELEASE-1.5.5/ROUGE-1.5.5.pl
2
2


AttributeError: 'int' object has no attribute 'decode'